In [ ]:
import pandas as pd

path = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\staging\can_awards_staging.csv"

df_preview = pd.read_csv(path, nrows=5)
df_preview



In [ ]:
cols_to_check = [
    "ID_NOTICE_CAN",
    "ANNO",
    "DT_DISPATCH",
    "CAE_NAME",
    "NUMBER_OFFERS",
    "NUMBER_TENDERS"
]

[c for c in cols_to_check if c in df_preview.columns]


In [ ]:
[col for col in df_preview.columns if col.strip().upper() == "ANNO"]


In [ ]:
import pandas as pd

path = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\staging\can_awards_staging.csv"

use_cols = [
    "ID_NOTICE_CAN",
    "DT_DISPATCH",
    "ANNO",
    "CAE_NAME",
    "NUMBER_OFFERS"
]

df_stage = pd.read_csv(path, usecols=use_cols)

df_stage["award_year"] = pd.to_datetime(
    df_stage["DT_DISPATCH"], errors="coerce"
).dt.year

df_stage.head()


In [ ]:
# mostra tutte le colonne che contengono 'ANN' (gestisce anche spazi/varianti)
[c for c in df_preview.columns if "ANN" in c.upper()]


In [ ]:
import pandas as pd

path = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\staging\can_awards_staging.csv"

use_cols = [
    "ID_NOTICE_CAN",
    "DT_DISPATCH",
    "CAE_NAME",
    "NUMBER_OFFERS"
]

df_stage = pd.read_csv(
    path,
    usecols=use_cols,
    nrows=10
)

df_stage["award_year"] = pd.to_datetime(
    df_stage["DT_DISPATCH"],
    errors="coerce",
    dayfirst=True
).dt.year

df_stage


In [ ]:
import pandas as pd

path = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\staging\can_awards_staging.csv"

use_cols = ["ID_NOTICE_CAN", "DT_DISPATCH", "CAE_NAME", "NUMBER_OFFERS"]

reader = pd.read_csv(
    path,
    usecols=use_cols,
    chunksize=10,      # legge solo il primo blocco
    dtype=str          # evita inferenza tipi
)

df_stage = next(reader)

df_stage["award_year"] = pd.to_datetime(
    df_stage["DT_DISPATCH"],
    errors="coerce",
    dayfirst=True
).dt.year

df_stage


In [ ]:
# Controlli minimi su sample (10 righe)
print("Null DT_DISPATCH:", df_stage["DT_DISPATCH"].isna().sum())
print("Null award_year:", df_stage["award_year"].isna().sum())

print("Range award_year (sample):", df_stage["award_year"].min(), "-", df_stage["award_year"].max())

print("Null ID_NOTICE_CAN:", df_stage["ID_NOTICE_CAN"].isna().sum())


In [ ]:
df_stage.loc[df_stage["award_year"] == 2017, ["DT_DISPATCH", "award_year"]]


In [ ]:
# Filtro temporale sul sample
df_stage_filtered = df_stage[df_stage["award_year"] >= 2018]

print("Righe prima:", len(df_stage))
print("Righe dopo:", len(df_stage_filtered))

df_stage_filtered


In [ ]:
import pandas as pd

path_in = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\staging\can_awards_staging.csv"
path_out = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\staging\can_awards_staging_clean.csv"

use_cols = ["ID_NOTICE_CAN", "DT_DISPATCH", "CAE_NAME", "NUMBER_OFFERS"]

chunks = pd.read_csv(
    path_in,
    usecols=use_cols,
    chunksize=200_000,
    dtype=str
)

first = True

for chunk in chunks:
    chunk["award_year"] = pd.to_datetime(
        chunk["DT_DISPATCH"],
        errors="coerce",
        dayfirst=True
    ).dt.year

    chunk = chunk[chunk["award_year"] >= 2018]

    chunk.to_csv(
        path_out,
        mode="w" if first else "a",
        header=first,
        index=False
    )
    first = False

print("Staging completo salvato.")


In [ ]:
import pandas as pd

# Percorsi input/output
path_in = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\staging\can_awards_staging_clean.csv"
path_out = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\fact_contract_awards.csv"

# Colonne necessarie per la FACT
use_cols = [
    "ID_NOTICE_CAN",   # identificativo contratto
    "DT_DISPATCH",     # data di aggiudicazione
    "CAE_NAME",        # fornitore
    "NUMBER_OFFERS"    # numero offerte
]

# Lettura a chunk per non saturare la memoria
chunks = pd.read_csv(
    path_in,
    usecols=use_cols,
    chunksize=200_000,
    dtype=str
)

first = True

for chunk in chunks:
    # Rinominare colonne in modo standard (data mart)
    chunk = chunk.rename(columns={
        "ID_NOTICE_CAN": "contract_id",
        "DT_DISPATCH": "award_date",
        "CAE_NAME": "supplier_name",
        "NUMBER_OFFERS": "num_offers"
    })

    # Derivare l'anno dalla data (asse temporale ufficiale)
    chunk["award_year"] = pd.to_datetime(
        chunk["award_date"],
        errors="coerce",
        dayfirst=True
    ).dt.year

    # Scrittura progressiva della FACT
    chunk.to_csv(
        path_out,
        mode="w" if first else "a",
        header=first,
        index=False
    )
    first = False

print("FACT table creata:", path_out)


In [ ]:
import pandas as pd

path_fact = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\fact_contract_awards.csv"

# Leggiamo solo un sample per controllare struttura e valori
df_fact_sample = pd.read_csv(path_fact, nrows=20, dtype=str)

print("Colonne:", list(df_fact_sample.columns))
print("Righe sample:", len(df_fact_sample))

print("Null contract_id:", df_fact_sample["contract_id"].isna().sum())
print("Null award_date:", df_fact_sample["award_date"].isna().sum())
print("Null supplier_name:", df_fact_sample["supplier_name"].isna().sum())

print("award_year min-max (sample):", df_fact_sample["award_year"].min(), "-", df_fact_sample["award_year"].max())

df_fact_sample.head()


In [ ]:
import pandas as pd

# Percorsi
path_fact = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\fact_contract_awards.csv"
path_dim = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\dim_supplier.csv"

# Leggiamo solo la colonna necessaria (efficiente)
suppliers = pd.read_csv(
    path_fact,
    usecols=["supplier_name"],
    dtype=str
)

# Rimuoviamo duplicati e valori nulli
suppliers = suppliers.dropna().drop_duplicates().reset_index(drop=True)

# Chiave surrogata
suppliers["supplier_id"] = suppliers.index + 1

# Riordino colonne (standard)
suppliers = suppliers[["supplier_id", "supplier_name"]]

# Salvataggio dimensione
suppliers.to_csv(path_dim, index=False)

print("dim_supplier creata:", path_dim)
print("Numero fornitori:", len(suppliers))


In [ ]:
import pandas as pd

# Percorsi
path_fact_in = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\fact_contract_awards.csv"
path_dim_supplier = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\dim_supplier.csv"
path_fact_out = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\fact_contract_awards_fk.csv"

# Carichiamo dimensione fornitori (chiave surrogata)
dim_supplier = pd.read_csv(path_dim_supplier, dtype=str)

# Prepariamo la FACT a chunk per non saturare la RAM
chunks = pd.read_csv(
    path_fact_in,
    chunksize=200_000,
    dtype=str
)

first = True

for chunk in chunks:
    # Join FACT -> DIM su supplier_name
    chunk = chunk.merge(
        dim_supplier,
        on="supplier_name",
        how="left"
    )

    # Selezioniamo solo le colonne finali della FACT
    chunk = chunk[[
        "contract_id",
        "award_date",
        "award_year",
        "supplier_id",
        "num_offers"
    ]]

    # Scrittura progressiva
    chunk.to_csv(
        path_fact_out,
        mode="w" if first else "a",
        header=first,
        index=False
    )
    first = False

print("FACT aggiornata con supplier_id:", path_fact_out)


In [ ]:
import pandas as pd

path_fact_fk = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\fact_contract_awards_fk.csv"

df_fk_sample = pd.read_csv(path_fact_fk, nrows=20000, dtype=str)

null_fk = df_fk_sample["supplier_id"].isna().sum()

print("Sample rows:", len(df_fk_sample))
print("Null supplier_id (sample):", null_fk)


In [ ]:
import pandas as pd

# === Percorsi input/output ===
path_fact_fk = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\fact_contract_awards_fk.csv"
path_dim_date = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\dim_date.csv"

# === Leggiamo solo la colonna data dalla FACT (efficiente) ===
dates = pd.read_csv(
    path_fact_fk,
    usecols=["award_date"],
    dtype=str
)

# === Convertiamo la data (formato europeo day/month/year) ===
dates["award_date"] = pd.to_datetime(
    dates["award_date"],
    errors="coerce",
    dayfirst=True
)

# === Teniamo solo date valide e uniche ===
dates = dates.dropna().drop_duplicates().sort_values("award_date").reset_index(drop=True)

# === Creiamo chiave surrogata per la dimensione date ===
dates["date_id"] = dates.index + 1

# === Deriviamo attributi calendario (utile per analisi in BI) ===
dates["year"] = dates["award_date"].dt.year
dates["month"] = dates["award_date"].dt.month
dates["quarter"] = dates["award_date"].dt.quarter

# === Riordiniamo le colonne in modo standard ===
dim_date = dates[["date_id", "award_date", "year", "quarter", "month"]]

# === Salviamo la dimensione date ===
dim_date.to_csv(path_dim_date, index=False)

print("dim_date creata:", path_dim_date)
print("Numero date uniche:", len(dim_date))


In [ ]:
import pandas as pd

# === Percorsi input/output ===
path_fact_in = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\fact_contract_awards_fk.csv"
path_dim_date = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\dim_date.csv"
path_fact_out = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\fact_contract_awards_fk2.csv"

# === Carichiamo la dimensione date ===
dim_date = pd.read_csv(path_dim_date, dtype=str)

# Nota: in dim_date 'award_date' è salvata come stringa ISO/standard dal CSV.
# Per un join robusto, normalizziamo le date anche nella FACT a un formato coerente.

# === Lettura della FACT a chunk per non saturare la RAM ===
chunks = pd.read_csv(
    path_fact_in,
    chunksize=200_000,
    dtype=str
)

first = True

for chunk in chunks:
    # === Normalizziamo award_date nella FACT (dayfirst europeo) ===
    chunk["award_date"] = pd.to_datetime(
        chunk["award_date"],
        errors="coerce",
        dayfirst=True
    ).dt.strftime("%Y-%m-%d")

    # === Normalizziamo award_date nella dim_date allo stesso formato ===
    dim_date["award_date"] = pd.to_datetime(
        dim_date["award_date"],
        errors="coerce"
    ).dt.strftime("%Y-%m-%d")

    # === Join FACT -> DIM su award_date normalizzata ===
    chunk = chunk.merge(
        dim_date[["date_id", "award_date"]],
        on="award_date",
        how="left"
    )

    # === Selezioniamo la FACT finale con chiavi ===
    chunk = chunk[[
        "contract_id",
        "date_id",
        "award_year",
        "supplier_id",
        "num_offers"
    ]]

    # === Scriviamo progressivamente su file ===
    chunk.to_csv(
        path_fact_out,
        mode="w" if first else "a",
        header=first,
        index=False
    )
    first = False

print("FACT aggiornata con date_id:", path_fact_out)


In [ ]:
import pandas as pd

# === Percorso della FACT con date_id ===
path_fact_fk2 = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\fact_contract_awards_fk2.csv"

# === Leggiamo un sample per verificare che date_id sia sempre valorizzato ===
df_fk2_sample = pd.read_csv(path_fact_fk2, nrows=20000, dtype=str)

# === Conteggio null sulla foreign key date_id ===
null_date_fk = df_fk2_sample["date_id"].isna().sum()

print("Sample rows:", len(df_fk2_sample))
print("Null date_id (sample):", null_date_fk)
print("Colonne presenti:", list(df_fk2_sample.columns))


In [ ]:
import pandas as pd

# === Percorso del file di staging originale (ha tutte le colonne) ===
path_stage = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\staging\can_awards_staging_clean.csv"

# === Leggiamo SOLO l'header (0 righe) per ottenere i nomi colonne senza caricare dati ===
cols = list(pd.read_csv(path_stage, nrows=0).columns)

# === Cerchiamo colonne che contengono parole chiave tipiche di "buyer" e "cpv" ===
buyer_like = [c for c in cols if "BUY" in c.upper() or "ACQUI" in c.upper() or "AUTH" in c.upper()]
cpv_like = [c for c in cols if "CPV" in c.upper()]

print("Buyer-like columns:", buyer_like)
print("CPV-like columns:", cpv_like)


In [ ]:
import pandas as pd

# === Percorsi input/output ===
path_fact_atomic = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\fact_contract_awards_fk2.csv"
path_fact_contract = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\mart\fact_contract_level.csv"

# === Leggiamo la FACT atomica ===
fact = pd.read_csv(path_fact_atomic, dtype=str)

# === Convertiamo num_offers a numerico per aggregare ===
fact["num_offers"] = pd.to_numeric(fact["num_offers"], errors="coerce")

# === Aggregazione a livello contratto ===
fact_contract = (
    fact
    .groupby(["contract_id", "date_id", "award_year", "supplier_id"], as_index=False)
    .agg({
        "num_offers": "max"   # scelta conservativa e difendibile
    })
)

# === Salvataggio ===
fact_contract.to_csv(path_fact_contract, index=False)

print("FACT_CONTRACT_LEVEL creata:", path_fact_contract)
print("Righe:", len(fact_contract))


In [ ]:
import pandas as pd

# === Percorso file staging completo ===
path_stage = r"C:\Users\karin\Portfolio\progetto2\eu-procurement-analytics\data\staging\can_awards_staging_clean.csv"

# === Leggiamo SOLO i nomi delle colonne (0 righe) ===
cols = list(pd.read_csv(path_stage, nrows=0).columns)

# === Parole chiave estese per BUYER / AUTHORITY ===
buyer_keywords = [
    "BUY", "AUTH", "AUTHORITY", "CA", "CONTRACTING", "ORG", "ORGANISATION",
    "ENTE", "ADMIN", "AGENCY"
]

# === Parole chiave estese per CPV / CATEGORY ===
cpv_keywords = [
    "CPV", "CLASS", "CATEGORY", "CAT", "MAIN", "OBJECT", "CODE"
]

buyer_like = [c for c in cols if any(k in c.upper() for k in buyer_keywords)]
cpv_like = [c for c in cols if any(k in c.upper() for k in cpv_keywords)]

print("Buyer-like columns:")
for c in buyer_like:
    print(" -", c)

print("\nCPV-like columns:")
for c in cpv_like:
    print(" -", c)
